コンペ概要

どのような金融市場でも、成功するためには、確かな投資対象を見極めることが必要です。株式やデリバティブが過小評価されている場合は、購入することが理にかなっています。割高であれば、売るべきでしょう。このような金融の判断は、これまで専門家が手作業で行ってきましたが、テクノロジーの進歩により、個人投資家にも新たな機会が訪れました。特にデータサイエンティストは、学習したモデルからの予測に基づいてプログラム上で意思決定を行う、定量取引に興味を持つかもしれません。

金融市場を分析し、投資戦略を策定するために使用される既存の定量取引の取り組みはたくさんあります。このような戦略を立て、実行するためには、過去とリアルタイムの両方のデータが必要ですが、特に個人投資家にとっては入手が困難です。今回のコンペティションでは、日本市場の金融データを提供することで、個人投資家が市場を最大限に分析することを可能にします。

株式会社日本取引所グループ（JPX）は、世界最大級の証券取引所である東京証券取引所、デリバティブ取引所の大阪取引所、東京工業品取引所を運営する持株会社です。本コンペティションは、JPXが主催し、AI技術企業のアルパカジャパン株式会社が協賛しています。

このコンペティションでは、学習段階が終了した後、実際の将来のリターンに対してモデルを比較します。コンペティションでは、予測対象銘柄（約2,000銘柄）からポートフォリオを構築します。具体的には、各参加者が期待リターンの高い銘柄から低い銘柄へと順位をつけ、上位200銘柄と下位200銘柄のリターンの差で評価されます。参加者は、日本市場の株式情報や過去の株価などの金融データにアクセスし、モデルの訓練と検証を行います。

受賞したモデルはすべて公開され、他の参加者は優れたモデルから学ぶことができます。また、優れたモデルは、クオンツ取引を実践したい人など、個人投資家の市場に対する関心を高める可能性があります。同時に、プログラムによる投資手法やポートフォリオ分析について自分なりの知見を得ることができ、日本市場に親近感を持つことができるかもしれません。

www.DeepL.com/Translator（無料版）で翻訳しました。

**参考**<br>
(1) コンペ評価指標<br>
   https://www.kaggle.com/code/smeitoma/jpx-competition-metric-definition/notebook

In [ ]:
!pip install japanize_matplotlib

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor
import optuna.integration.lightgbm as lgb
import seaborn as sns
import japanize_matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

## 過去のNotebook


# メインデータ

In [ ]:
prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv", parse_dates=["Date"])
print(prices.shape)
prices.head()

列の意味<br>
(1) RowId: 日付_銘柄。ユニークID<br>
(2) Date：日付<br>
(3) SecuritiesCode: 銘柄コード。2000種類存在。<br>
(4) Open:始値<br>
(5) High:高値<br>
(6) Low :安値<br>
(7) Close:終値<br>
(8) Volume:出来高<br>
(9) AdjustmentFactor：分割／併合による株価の変動。終値から次の日の始まりまでの間に補正される。<br>
(10) ExpectedDividend:予想配当金額。<br>
(11) SupervisionFlag: 監理・整理銘柄フラグ。上場廃止リスク高い。<br>
(12) Target:目標変数。１日後と2日後の差額から得られる収益率<br>

In [ ]:
## 欠損値確認。目標変数（Target）
print(prices.isnull().sum())

In [ ]:
#取引が成立しなかった=出来高0を意味する
(prices["Open"].isnull() == (prices["Volume"]==0)).all()

In [ ]:
##2000銘柄×98日分存在。ただし、1413では4日分、8806には3日間の欠損がある
prices["SecuritiesCode"].value_counts().sort_values()

In [ ]:
# 最後の4日間、3日間が欠損。途中抜けはなし
pd.pivot(prices,index="Date", columns="SecuritiesCode", values="Close")[[1413,8806]]

## Targetを確認
https://www.kaggle.com/code/smeitoma/jpx-competition-metric-definition/notebook<br>
より、<br>
Target = (翌々日終値 - 翌日終値)/翌日終値<br>
つまり、翌日の引成で買って、翌々日に引成で売った場合のリターン<br>

In [ ]:
## 確認。
print(prices["Target"].head())
print(prices.groupby("SecuritiesCode")["Close"].apply(lambda s:(s.shift(-2) - s.shift(-1))/s.shift(-1)).head())

In [ ]:
# ほぼ誤差0なのでおそらく正しいはず。AdjustmentFactorで一応補正.
# 率にして10^-4　以下なので情報落ちとか？
df = prices.copy()
df["Close_adj"] = df.groupby("SecuritiesCode").apply(lambda d:d["Close"]/d["AdjustmentFactor"].cumprod().shift().fillna(1)).reset_index("SecuritiesCode",drop=True)
df["R"] = df.groupby("SecuritiesCode")["Close_adj"].apply(lambda s:(s.shift(-2) - s.shift(-1))/s.shift(-1))
(df["R"]-df["Target"]).describe()

In [ ]:
## 一応誤差が10-^5以上のもの確認。
## 一部の銘柄に大きな偏り。4923,4826, 9104, 7723, 8545, 1390, 9441
tmp = df[abs(df["R"]-df["Target"])>0.00001]
display(tmp)
tmp["SecuritiesCode"].value_counts()

In [ ]:
## 4923についてTargetと算出したR（収益率）を比較。ほぼ重なっているため無視して良いと判断する
d = df[df["SecuritiesCode"]==4923].copy()

sns.lineplot(x=d.Date, y=d.R, label="収益率")
sns.lineplot(x=d.Date, y=d.Target, label="Target")
plt.show()
sns.lineplot(x=d.Date, y=d.Close, label="株価")
sns.lineplot(x=d.Date, y=d.Close.rolling(5).mean(), label="5日移動平均線")
sns.lineplot(x=d.Date, y=d.Close.rolling(25).mean(), label="25日移動平均線")
sns.lineplot(x=d.Date, y=d.Close.rolling(75).mean(), label="75日移動平均線")